In [1]:
#Print current hardware information (variable in Colab) 

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Feb 23 11:29:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
#@title
import os
import sys
import tensorflow.compat.v1 as tf

Read from my Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Move to "_modified-EfficientDet" folder and install requirements

In [4]:
os.chdir('drive/MyDrive/_modified-EfficientDet')
sys.path.append('.')
!pip install -r requirements.txt
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-rvynatrq
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-rvynatrq
     |████████████████████████████████| 6.4 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 70.2 MB/s 
     |████████████████████████████████| 1.1 MB 58.7 MB/s 
     |████████████████████████████████| 120 kB 73.8 MB/s 
     |████████████████████████████████| 234 kB 62.0 MB/s 
     |████████████████████████████████| 462 kB 71.2 MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264359 sha256=0b8996ed995cbfabe6548bd3f0d8932af081f976fe92ed1d3e1548705140996b
  Stored in directory: /tmp/pip-ephem-wheel-cache-t1nhgbe2/wheels/e2/6b/1d/344ac773c7495ea0b85eb228bc66daec7400a143a92d36b7b1
Successfully built pycocotools
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyY

In [15]:
MODEL = 'efficientdet-d3_1-5'  #@param


# Prepare image and visualization settings.
image_url =  'https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png'#@param
image_name = 'img.png' #@param
!wget {image_url} -O img.png
import os
img_path = os.path.join(os.getcwd(), 'img.png')

min_score_thresh = 0.35  #@param
max_boxes_to_draw = 200  #@param
line_thickness =   2#@param

import PIL
# Get the largest of height/width and round to 128.
image_size = max(PIL.Image.open(img_path).size)

--2022-02-23 12:13:20--  https://user-images.githubusercontent.com/11736571/77320690-099af300-6d37-11ea-9d86-24f14dc2d540.png
Resolving user-images.githubusercontent.com (user-images.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to user-images.githubusercontent.com (user-images.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4080549 (3.9M) [image/png]
Saving to: ‘img.png’

img.png             100%[===================>]   3.89M  --.-KB/s    in 0.08s   

2022-02-23 12:13:21 (48.8 MB/s) - ‘img.png’ saved [4080549/4080549]



In [16]:
# image_size = 512x512 for d0, 640x640 for d1, 768x768 for d2 and 896x896 for d3.

!python main.py --mode=eval  \
    --model_name={MODEL}  --model_dir='tmp_model_dir/efficientdet-d3_1-5-scratch-Trash'  \
    --val_file_pattern='tfrecord_val/TrashICRA19-train-1-of-19-dark120.tfrecord'  --hparams="num_classes=3,moving_average_decay=0,image_size=896x896,mixed_precision=true"  

I0223 12:14:38.025500 139967394858880 main.py:228] {'name': 'efficientdet-d3_1-5', 'act_type': 'swish', 'image_size': (896, 896), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': None, 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 3, 'seg_num_classes': 3, 'heads': ['object_detection'], 'skip_crowd_during_training': True, 'label_map': None, 'max_instances_per_image': 100, 'regenerate_source_id': False, 'min_level': 3, 'max_level': 7, 'num_scales': 3, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_scale': 4.0, 'is_training_bn': True, 'momentum': 0.9, 'optimizer': 'sgd', 'learning_rate': 0.08, 'lr_warmup_init': 0.008, 'lr_warmup_epoch': 1.0, 'first_lr_drop_epoch': 200.0, 'second_lr_drop_epoch': 250.0, 'poly_lr_power': 0.9, 'clip_gradients_norm': 10.0, 'num_epochs': 300, 'data_format': 'channels_last', 'mean_rgb': [123.675, 116.28, 103.53], 'stddev_rgb': [58.395, 57.120000000000005, 57.375], 'scale_range': False

In [17]:
# image_size = 512x512 for d0, 640x640 for d1, 768x768 for d2 and 896x896 for d3.

!python main.py --mode=eval  \
    --model_name={MODEL}  --model_dir='tmp_model_dir/efficientdet-d3_1-5-scratch-Trash'  \
    --val_file_pattern='tfrecord_val/TrashICRA19-train-1-of-19-dark120_enhanced_l2uwe.tfrecord'  --hparams="num_classes=3,moving_average_decay=0,image_size=896x896,mixed_precision=true"  

I0223 12:16:08.177490 139964292921216 main.py:228] {'name': 'efficientdet-d3_1-5', 'act_type': 'swish', 'image_size': (896, 896), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': None, 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 3, 'seg_num_classes': 3, 'heads': ['object_detection'], 'skip_crowd_during_training': True, 'label_map': None, 'max_instances_per_image': 100, 'regenerate_source_id': False, 'min_level': 3, 'max_level': 7, 'num_scales': 3, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_scale': 4.0, 'is_training_bn': True, 'momentum': 0.9, 'optimizer': 'sgd', 'learning_rate': 0.08, 'lr_warmup_init': 0.008, 'lr_warmup_epoch': 1.0, 'first_lr_drop_epoch': 200.0, 'second_lr_drop_epoch': 250.0, 'poly_lr_power': 0.9, 'clip_gradients_norm': 10.0, 'num_epochs': 300, 'data_format': 'channels_last', 'mean_rgb': [123.675, 116.28, 103.53], 'stddev_rgb': [58.395, 57.120000000000005, 57.375], 'scale_range': False

In [18]:
# image_size = 512x512 for d0, 640x640 for d1, 768x768 for d2 and 896x896 for d3.

!python main.py --mode=eval  \
    --model_name={MODEL}  --model_dir='tmp_model_dir/efficientdet-d3_1-5-scratch-Trash'  \
    --val_file_pattern='tfrecord_val/TrashICRA19-train-1-of-19-dark120_enhanced_c40.tfrecord'  --hparams="num_classes=3,moving_average_decay=0,image_size=896x896,mixed_precision=true"  

I0223 12:17:25.850390 139689178179456 main.py:228] {'name': 'efficientdet-d3_1-5', 'act_type': 'swish', 'image_size': (896, 896), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': None, 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 3, 'seg_num_classes': 3, 'heads': ['object_detection'], 'skip_crowd_during_training': True, 'label_map': None, 'max_instances_per_image': 100, 'regenerate_source_id': False, 'min_level': 3, 'max_level': 7, 'num_scales': 3, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_scale': 4.0, 'is_training_bn': True, 'momentum': 0.9, 'optimizer': 'sgd', 'learning_rate': 0.08, 'lr_warmup_init': 0.008, 'lr_warmup_epoch': 1.0, 'first_lr_drop_epoch': 200.0, 'second_lr_drop_epoch': 250.0, 'poly_lr_power': 0.9, 'clip_gradients_norm': 10.0, 'num_epochs': 300, 'data_format': 'channels_last', 'mean_rgb': [123.675, 116.28, 103.53], 'stddev_rgb': [58.395, 57.120000000000005, 57.375], 'scale_range': False

In [19]:
# image_size = 512x512 for d0, 640x640 for d1, 768x768 for d2 and 896x896 for d3.

!python main.py --mode=eval  \
    --model_name={MODEL}  --model_dir='tmp_model_dir/efficientdet-d3_1-5-scratch-Trash'  \
    --val_file_pattern='tfrecord_val/TrashICRA19-train-1-of-19-dark120_enhanced_c80.tfrecord'  --hparams="num_classes=3,moving_average_decay=0,image_size=896x896,mixed_precision=true"  

I0223 12:18:32.331751 140618633959296 main.py:228] {'name': 'efficientdet-d3_1-5', 'act_type': 'swish', 'image_size': (896, 896), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.1, 'jitter_max': 2.0, 'autoaugment_policy': None, 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 3, 'seg_num_classes': 3, 'heads': ['object_detection'], 'skip_crowd_during_training': True, 'label_map': None, 'max_instances_per_image': 100, 'regenerate_source_id': False, 'min_level': 3, 'max_level': 7, 'num_scales': 3, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_scale': 4.0, 'is_training_bn': True, 'momentum': 0.9, 'optimizer': 'sgd', 'learning_rate': 0.08, 'lr_warmup_init': 0.008, 'lr_warmup_epoch': 1.0, 'first_lr_drop_epoch': 200.0, 'second_lr_drop_epoch': 250.0, 'poly_lr_power': 0.9, 'clip_gradients_norm': 10.0, 'num_epochs': 300, 'data_format': 'channels_last', 'mean_rgb': [123.675, 116.28, 103.53], 'stddev_rgb': [58.395, 57.120000000000005, 57.375], 'scale_range': False

REFERENCES

[1] F. Zocco et al., Towards More Efficient EfficientDets and Low-Light Real-Time Marine Debris Detection, https://arxiv.org/pdf/2203.07155.pdf

[2] https://github.com/google/automl/blob/master/efficientdet/tutorial.ipynb

[3] https://openaccess.thecvf.com/content_CVPR_2020/papers/Tan_EfficientDet_Scalable_and_Efficient_Object_Detection_CVPR_2020_paper.pdf